# Simple Neural Network

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

## DataSet
For data set i will use a data generetion function

In [4]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [8]:
X, y = load_coffee_data()
print(X.shape, y.shape)

(200, 2) (200, 1)


## Normalize Data

In [13]:
print(f'Temperature Max, Min pre normalization: {np.max(X[:,0])}, {np.min(X[:,0])}')
print(f'Duration Max, Min pre normalization: {np.max(X[:,1])}, {np.min(X[:,1])}')
norm_l = tf.keras.layers.Normalization(axis=1)
norm_l.adapt(X) #learns mean, variance
Xn = norm_l(X)
print(f'Temperature Max, Min post normalization: {np.max(Xn[:,0])}, {np.min(Xn[:,0])}')
print(f'Duration Max, Min post normalization: {np.max(Xn[:,1])}, {np.min(Xn[:,1])}')

Temperature Max, Min pre normalization: 284.99434167453603, 151.32372212128612
Duration Max, Min pre normalization: 15.454206928801511, 11.512747082472469
Temperature Max, Min post normalization: 1.659753680229187, -1.6852186918258667
Duration Max, Min post normalization: 1.793238878250122, -1.6996712684631348


Tile/copy our data to increase the training set size and reduce the number of training epochs.

In [14]:
Xt = np.tile(Xn,(1000, 1))
Yt = np.tile(y, (1000, 1))
print(Xt, Yt)

[[-0.8345551  -0.6528792 ]
 [ 1.0323051  -1.3851453 ]
 [ 0.3089391   0.8716251 ]
 ...
 [ 1.3767233  -0.29521284]
 [ 0.04860044 -0.5590083 ]
 [ 1.6597537  -0.62245744]] [[1.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


## Tensorflow Model
Let's build the "Coffee Roasting Network" described in lecture. There are two layers with sigmoid activations.

In [19]:
#  Установка фиксированного значения для генератора случайных чисел
#При каждом запуске этого кода с tf.random.set_seed(1234) вы будете получать один и тот же случайный тензор.
#Если убрать или изменить значение в set_seed,
# полученные случайные значения будут различаться при каждом запуске программы.
tf.random.set_seed(1234)

model = Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(3, activation='sigmoid', name ='layer1'),
    Dense(1, activation='sigmoid', name = 'layer2')
])

model.summary() #description of the network

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The parameter counts shown in the summary correspond to the number of elements in the weight and bias arrays as shown below.

In [20]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

L1 params =  9 , L2 params =  4


Let's examine the weights and biases Tensorflow has instantiated.  The weights $W$ should be of size (number of features in input, number of units in the layer) while the bias $b$ size should match the number of units in the layer:
- In the first layer with 3 units, we expect W to have a size of (2,3) and $b$ should have 3 elements.
- In the second layer with 1 unit, we expect W to have a size of (3,1) and $b$ should have 1 element.

In [23]:
W1, b1 = model.get_layer('layer1').get_weights()
W2, b2 = model.get_layer('layer2').get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[-0.84030825 -0.66032195  0.5632168 ]
 [ 0.04501367 -0.7248856  -1.0070844 ]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.14662266]
 [ 0.3569262 ]
 [-0.2701342 ]] 
b2(1,): [0.]
